In [6]:
import numpy as np
# import matplotlib.pyplot as plt
from scipy.misc import imread
import cv2 


def add_gaussian_noise(im,prop,varSigma):
    N = int(np.round(np.prod(im.shape)*prop))
    index = np.unravel_index(np.random.permutation(np.prod(im.shape))[1:N],im.shape)
    e = varSigma*np.random.randn(np.prod(im.shape)).reshape(im.shape)
    im2 = np.copy(im).astype('float')
    im2[index] += e[index]
    return im2

def add_saltnpeppar_noise(im,prop):
    N = int(np.round(np.prod(im.shape)*prop))
    index = np.unravel_index(np.random.permutation(np.prod(im.shape))[1:N],im.shape)
    im2 = np.copy(im)
    im2[index] = 1-im2[index]
    return im2

# proportion of pixels to alter    
prop = 0.7
varSigma = 0.1

# To not use cv2 just delete uncomment the codes below and comment out the codes using cv2 

im = cv2.imread('./figures/pug.jpg',0)
im = im/255
cv2.imshow('greyscale',im)
cv2.imwrite('greyscale.jpg',im)

im2 = add_gaussian_noise(im,prop,varSigma)
gauss_noise = cv2.imshow('gaussian noise',im2)
cv2.imwrite('gaussian_noise.png', im2)

im3 = add_saltnpeppar_noise(im,prop)
snp_noise = cv2.imshow('salt-and-pepper noise', im3)
cv2.imwrite('snp_noise.png', im3)

cv2.waitKey(0)
cv2.destroyAllWindows()

# im = im/255
# fig = plt.figure()
# ax = fig.add_subplot(131)
# ax.imshow(im,cmap='gray')
# im2 = add_gaussian_noise(im,prop,varSigma)
# ax2 = fig.add_subplot(132)
# ax2.imshow(im2,cmap='gray')
# im3 = add_saltnpeppar_noise(im,prop)
# ax3 = fig.add_subplot(133)
# ax3.imshow(im3,cmap='gray')


ModuleNotFoundError: No module named 'cv2.cv2'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
from scipy.misc import imread

def add_gaussian_noise(im,prop,varSigma):
    N = int(np.round(np.prod(im.shape)*prop))
    index = np.unravel_index(np.random.permutation(np.prod(im.shape))[1:N],im.shape)
    e = varSigma*np.random.randn(np.prod(im.shape)).reshape(im.shape)
    im2 = np.copy(im).astype('float')
    im2[index] += e[index]
    return im2

def add_saltnpeppar_noise(im,prop):
    N = int(np.round(np.prod(im.shape)*prop))
    index = np.unravel_index(np.random.permutation(np.prod(im.shape))[1:N],im.shape)
    im2 = np.copy(im)
    im2[index] = 1-im2[index]
    return im2
    # proportion of pixels to alter
    
prop = 0.7
varSigma = 1
im = imageio.imread('grey_pug.jpg')
im = im/255
fig = plt.figure()
ax = fig.add_subplot(131)
ax.imshow(im,cmap='gray')
im2 = add_gaussian_noise(im,prop,varSigma)
print(type(im2))
ax2 = fig.add_subplot(132)
ax2.imshow(im2,cmap='gray')
print(type(ax2))
im3 = add_saltnpeppar_noise(im,prop)
ax3 = fig.add_subplot(133)
ax3.imshow(im3,cmap='gray')
print(im)
print(im2)
print(im3)

In [ ]:
print(im)
img_gray_arr = np.asarray(im)
x = np.rint(img_gray_arr)
img_mean = np.mean(x)
img_arr = np.copy(x)
img_arr[x<img_mean] = -1
img_arr[x>=img_mean] = 1


In [ ]:
print(img_gray_arr)
print(img_mean)

print(img_arr)
print(img_arr[0:10,0:10])
print(img_arr.shape)
plt.imshow(img_arr, cmap = 'gray')

In [ ]:
noisy_img_gray_arr = np.asarray(im2)
y = np.rint(noisy_img_gray_arr)
noisy_img_mean = np.mean(y)
noisy_img_arr = np.copy(y)
noisy_img_arr[y<noisy_img_mean] = -1
noisy_img_arr[y>=noisy_img_mean] = 1

plt.imshow(noisy_img_arr, cmap = 'gray')

print(noisy_img_arr)
print(type(noisy_img_arr))

In [ ]:
def percent_pixel_flipped(noisy_arr,denoised_arr):
    num_flipped = 0
    total_px = noisy_arr.shape[0]*noisy_arr.shape[1]
    for i in range(noisy_arr.shape[0]):
        for j in range(noisy_arr.shape[1]):
            if noisy_arr[i,j]!= denoised_arr[i,j]:
                num_flipped+=1
   
    percent_change = num_flipped*100./total_px
    return percent_change

print (percent_pixel_flipped(noisy_img_arr, img_arr))

In [ ]:

def check_limit(value, limit):
    if value<0:
        value=limit-1
    if value==limit:
        value=0
    return value


def energyfx(y_array, x_array, xval,yval, constant):
    h = constant[0]
    b = constant[1]
    n = constant[2]
    total_pixels = x_array.shape[0]*x_array.shape[1]
    energy = h * x_array[xval, yval] 
    energy += - n * x_array[xval, yval] * y_array[xval, yval]
    x_neighbour = [-1,1]
    y_neighbour = [-1,1]
    for i in x_neighbour:
        for j in y_neighbour:
            x_n = check_limit(xval +i,x_array.shape[0])
            y_n = check_limit(yval +j, x_array.shape[1])
            
            energy += -b*x_array[xval,yval]*x_array[x_n,y_n]
    energy = energy/total_pixels
    return energy

def calculate_total_energy(y_array,x_array,constant):
    energy = 0.
    for i in range(y_array.shape[0]):
        for j in range(y_array.shape[1]):
            energy += energyfx(y_array, x_array, i,j, constant)
    return energy

#this list is [h, beta,eta]
constant = [0,.1,.02]
hidden_image = np.copy(noisy_img_arr)
total_energy= calculate_total_energy(noisy_img_arr, hidden_image, constant)
print(total_energy)


In [ ]:
def icm_single_pixel(visible_arr, hidden_arr, px_x, px_y, total_energy, const_list):
    current_energy = energyfx(visible_arr, hidden_arr,px_x,px_y, const_list)
    other_energy = total_energy - current_energy
    #flip the pixel
    new_hidden_arr = np.copy(hidden_arr)
    if hidden_arr[px_x,px_y]==1:
        new_hidden_arr[px_x,px_y]=-1
    else:
        new_hidden_arr[px_x,px_y] = 1
    flipped_energy = energyfx(visible_arr, new_hidden_arr,px_x,px_y, const_list)
    #print current_energy, flipped_energy
    if flipped_energy < current_energy:
        should_flip = True
        total_energy = other_energy + flipped_energy
        hidden_arr = new_hidden_arr
        #print percent_pixel_flipped(hidden_arr, visible_arr)
    else:
        should_flip = False
    
    return (hidden_arr,should_flip,total_energy)
    #return (should_flip, hidden_arr, total_energy)

#main icm simulation
hidden_image = np.copy(noisy_img_arr)
energy_this_round = total_energy
print ("% Pixels flipped:", percent_pixel_flipped(hidden_image, img_arr))

for sim_round in range(10):
    for i in range(hidden_image.shape[0]):
        for j in range(hidden_image.shape[1]):
            hidden_image,should_flip,total_energy = icm_single_pixel(noisy_img_arr,hidden_image,i,j, total_energy,constant)
            
        #print percent_pixel_flipped(hidden_image, lena_arr)
    if (total_energy - energy_this_round) == 0:
        print ("Algorithm converged")
        break
    energy_this_round = total_energy
    print ("Total Energy:",total_energy)
    print ("% Pixels flipped:", percent_pixel_flipped(hidden_image, img_arr))

In [ ]:
plt.imshow(hidden_image, cmap='gray')
print(hidden_image)
print (percent_pixel_flipped(hidden_image, img_arr))